In [1]:
import requests
import pandas as pd
import json
from psaw import PushshiftAPI
from nltk import ngrams
api = PushshiftAPI()
from timeit import default_timer as timer

In [2]:
# function that gets comments dataframe for a post
def get_post(**kwargs):
    base_url = f"https://api.pushshift.io/reddit/search/submission/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    data = request.json().get("data")
    l = list(pd.DataFrame.from_records(data).id)
    return l

# function that grabs a post from a subreddit
def get_pushshift_data(data_type, **kwargs):

 
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    data = request.json().get("data")
    df = pd.DataFrame.from_records(data)[['author','subreddit','body','created_utc']]
    return df

#  function that counts the number of n-grams in a column of a reddit dataframe
def count_grams(df, i):
    df_body = df['body']
    
    grams_dict = {}
    for s in df_body:
        grams = ngrams(s.split(),i)

        for gram in grams:
            gram2 = ' '.join(gram)
            if gram2 in grams_dict:
                grams_dict[gram2] += 1
            else:
                grams_dict[gram2] = 1

    final_df = pd.DataFrame([grams_dict]).T

    return final_df

# collects gram counts from multiple subreddits
def collect_grams(subs,i):
    final_df = pd.DataFrame()
    error_list = []
    for s in subs:
        try:
            posts = get_post(subreddits= s, size = 50, after="30d")
            for p in posts:
                try:
                    df1 = get_pushshift_data(data_type='comment',id = p, after="30d", size = 50)
                    count_df = count_grams(df1,i)
                    count_df[str(i)+'-gram'] = count_df.index
                    count_df['subreddit'] = s
                    count_df = count_df.rename(columns={0:'count'})
                    final_df = pd.concat([count_df, final_df], ignore_index=True)
                except:
                    print(p)
                print("posts remaining:" + str(len(posts) - posts.index(p) - 1))
        except:
            print(s)
        print("Subs remaining:" + str(len(subs) - subs.index(s) - 1))
        
    
    return final_df

In [3]:
subs_df = pd.read_csv('list_of_subreddits.csv')
subs_list = [s[3:] for s in subs_df['Subreddits']]

subs_list
subs_3 = subs_list[:3]
subs_10 = subs_list[:10]
subs_50 = subs_list[:50]
subs_100 = subs_list[:100]

In [4]:
start = timer()
gram_count_1 = collect_grams(subs_3,1)
elapsed_time = timer() - start
print(elapsed_time)
gram_count_1.to_csv("gram_count_test_s3.csv")

posts remaining:49
posts remaining:48
posts remaining:47
posts remaining:46
posts remaining:45
posts remaining:44
posts remaining:43
posts remaining:42
posts remaining:41
posts remaining:40
posts remaining:39
posts remaining:38
posts remaining:37
posts remaining:36
posts remaining:35
posts remaining:34
posts remaining:33
posts remaining:32
posts remaining:31
posts remaining:30
posts remaining:29
posts remaining:28
posts remaining:27
posts remaining:26
posts remaining:25
posts remaining:24
posts remaining:23
posts remaining:22
posts remaining:21
posts remaining:20
posts remaining:19
posts remaining:18
posts remaining:17
posts remaining:16
posts remaining:15
posts remaining:14
posts remaining:13
posts remaining:12
posts remaining:11
posts remaining:10
posts remaining:9
posts remaining:8
posts remaining:7
posts remaining:6
posts remaining:5
posts remaining:4
posts remaining:3
posts remaining:2
posts remaining:1
posts remaining:0
Subs remaining:2
posts remaining:49
posts remaining:48
posts